In [13]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pandas.plotting import table
from matplotlib.table import table
from bs4 import BeautifulSoup
import base64
from io import BytesIO

file = "/workspaces/SPL/Middleput/points.xlsx"
points = pd.read_excel(file)
data = pd.read_excel(file)
# Setting option to display all rows
pd.set_option('display.max_rows', None)

In [14]:
Data_S1 = data[data["Season"] == 1]
Data_S2 = data[data["Season"] == 2]
data = data[data["Season"] == 2]

In [15]:
def compute_player_summaries(data):
    """
    Compute player summaries based on the provided data.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A summary dataframe containing computed metrics for each player.
    """
    # Define aggregation functions for the columns of interest
    aggregations = {
        'Date': 'count',
        'Penalty': 'sum',
        'Friend Referrals': 'sum',
        'Own Goals': 'sum',
        'Goals Conceded': 'sum',
        'Goals': ['mean', 'sum'],
        'Total Points': ['mean', 'sum'],
        'MVP': 'sum',
        'SPL Bonus': 'sum'
    }
    
    # Use groupby with multiple aggregation functions
    player_summary = data.groupby('Player').agg(aggregations)
    
    # Flatten hierarchical columns
    player_summary.columns = ['_'.join(col).strip() for col in player_summary.columns.values]
    
    # Rename columns for clarity
    columns_rename = {
        'Date_count': 'Games Played',
        'Penalty_sum': 'Penalties',
        'Friend Referrals_sum': 'Friend Referrals',
        'Own Goals_sum': 'Own Goals',
        'Goals Conceded_sum': 'Goals Conceded',
        'Goals_mean': 'Goals/Game',
        'Goals_sum': 'Total Goals',
        'Total Points_mean': 'Points/Game',
        'Total Points_sum': 'Total',
        'MVP_sum': 'MVP',
        'SPL Bonus_sum': 'SPL Bonus'
    }
    player_summary.rename(columns=columns_rename, inplace=True)
    
    # Calculate games won
    games_won = data[data['Team'] == data['Winning Team']].groupby('Player').size()
    player_summary['Games Won'] = games_won
    player_summary['Games Won'].fillna(0, inplace=True)
    
    # Calculate win ratio
    player_summary['Win Ratio'] = player_summary['Games Won'] / player_summary['Games Played']
    
    return player_summary

def calculate_cumulative_points_and_rank(data):
    """
    Calculate cumulative points and rank changes for each player.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A dataframe containing the 'Rank Change' for the latest game for each player.
    """
    # Work with a deep copy to avoid modifying the original dataframe
    data_copy = data.copy()
    
    # Calculate cumulative Total Points for each player after each game
    data_copy['Cumulative Points'] = data_copy.groupby('Player')['Total Points'].cumsum()
    
    # Determine the player's rank based on these cumulative points after each game
    data_copy['Rank'] = data_copy.groupby('Date')['Cumulative Points'].rank(method="first", ascending=False)
    
    # Sort data to ensure we process in chronological order for each player
    data_sorted = data_copy.sort_values(by=['Player', 'Date'])
    
    # Calculate the change in rank between each game for every player
    data_sorted['Rank Change'] = data_sorted.groupby('Player')['Rank'].diff().fillna(0)
    
    # Extract the latest rank change for each player
    latest_rank_change = data_sorted.groupby('Player').apply(lambda x: x.iloc[-1])['Rank Change']
    
    return latest_rank_change

In [16]:
def generate_summary(data):
    # Integrate functions to generate the final summary
    player_summary = compute_player_summaries(data)
    latest_rank_change = calculate_cumulative_points_and_rank(data)

    # Add 'Rank Change' to the summary
    player_summary['Rank Change'] = latest_rank_change

    # Fill NaN values with 0
    player_summary = player_summary.fillna(0)

    # Convert specific columns to integer type
    cols_to_int = ['Games Won', 'MVP', 'SPL Bonus', 'Rank Change']
    player_summary[cols_to_int] = player_summary[cols_to_int].astype(int)

    # Calculate the overall rank based on the 'Total Points'
    player_summary['Rank'] = player_summary['Total'].rank(method="min", ascending=False).astype(int)

    # Reorder columns to have 'Rank' at the front
    column_order = ['Rank'] + [col for col in player_summary if col != 'Rank']
    final_summary = player_summary[column_order]

    # Sorting the final_summary DataFrame by 'Rank' in ascending order
    sorted_summary = final_summary.sort_values(by='Rank')

    # Reordering the columns as specified
    desired_column_order = [
        "Player", "Rank", "Games Played", "Games Won", "Win Ratio", "Penalties", 
        "Friend Referrals", "Own Goals", "Goals Conceded", "MVP", "SPL Bonus",
        "Goals/Game", "Total Goals", "Points/Game", 
        "Total", "Rank Change"
    ]

    for col in sorted_summary.columns:
        if sorted_summary[col].dtype == 'float64':
            sorted_summary[col] = sorted_summary[col].round(2)
    sorted_summary['Win Ratio'] = (sorted_summary['Win Ratio'] * 100).round(0).astype(int).astype(str) + '%'
    
    # Reset index to get the "Player" column and then reorder columns
    sorted_summary = sorted_summary.reset_index()
    sorted_summary = sorted_summary[desired_column_order]
    
    return sorted_summary

In [17]:
season1 = generate_summary(Data_S2)
sorted_summary = season1

In [18]:
season1

Player  Rank  Games Played  Games Won Win Ratio  Penalties  \
0    Cameron McAinsh     1             2          1       50%          0   
1              Cerro     2             2          1       50%          0   
2  Damiano Barbanera     2             2          1       50%          0   
3              Mazzu     4             2          1       50%          0   
4     Ludovico Righi     5             2          1       50%          0   
5   Andrea Scalambra     6             2          1       50%          0   
6  Federico Paolucci     6             2          1       50%          0   
7         Davide Ang     6             2          1       50%          0   
8       Wissam Rahal     9             2          1       50%          0   

   Friend Referrals  Own Goals  Goals Conceded  MVP  SPL Bonus  Goals/Game  \
0                 0          0              11    0          0         3.0   
1                 0          0              12    0          0         3.0   
2                 0          0              11    0          0         1.0   
3                 0          0              12    0          0         2.0   
4                 0          0              11    0          0         0.0   
5                 0          0              12    0          0         0.0   
6                 0          0              12    0          0         0.0   
7                 0          0              12    0          0         0.0   
8                 0          0              11    0          0         0.0   

   Total Goals  Points/Game  Total  Rank Change  
0            6         13.5     27            0  
1            6         12.5     25           -2  
2            2         12.5     25            0  
3            4         11.5     23           -2  
4            0         10.5     21            2  
5            0          9.5     19           -1  
6            0          9.5     19           -1  
7            0          9.5     19           -1  
8            0          8.0     16            5

In [19]:
# Save the games_df DataFrame as an Excel file
sorted_summary_excel = "/workspaces/SPL/Middleput/sorted_summary.xlsx"
sorted_summary.to_excel(sorted_summary_excel, index=False)

In [20]:
player_data = data[data['Player'] == "Cameron McAinsh"]

In [21]:
def mpl_to_html(fig):
    """
    Convert a Matplotlib Figure object into a Base64-encoded PNG and return an HTML string.
    """
    buf = BytesIO()
    fig.savefig(buf, format="png")
    buf.seek(0)
    image_base64 = base64.b64encode(buf.read()).decode('utf-8').replace('\n', '')
    buf.close()
    return f"<html><body><img src='data:image/png;base64,{image_base64}' /></body></html>"

In [22]:
def player_graphs_with_summary_updated(player_name: str):
    """
    Display player summary in a table and generate four graphs for the specified player:
    - Total Points per Gameweek
    - Number of Goals per Gameweek
    - Pie chart showing Wins, Losses, and Draws
    - Line chart showing Defensive score and Midfield score over Gameweek

    Parameters:
    - player_name (str): Name of the player
    """
    
    # Filter the original data for the specified player
    player_data = data[data['Player'] == player_name]
    
    # Filter the sorted_summary data for the specified player
    player_summary = sorted_summary[sorted_summary['Player'] == player_name]
    
    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return
    
    # Set up the plot environment
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), gridspec_kw={"height_ratios": [0.2, 0.4, 0.4]})
    
    # Display the player's summary data in a table format
    # Remove the index from the table and use the player's name as the title
    axes[0, 0].axis("off")
    axes[0, 1].axis("off")
    ax_center = fig.add_subplot(3, 2, (1, 2))
    ax_center.axis("off")
    
    player_summary = player_summary.drop(columns=['Player']).T
    player_summary.columns = [player_name]
    cell_text = player_summary.reset_index().values.tolist()
    tbl = table(ax_center, cellText=cell_text, loc="center", cellLoc="center")
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(10)
    tbl.scale(1.0, 1.2)
    
    # Total Points per Gameweek
    sns.lineplot(x='Gameweek', y='Total Points', data=player_data, ax=axes[1, 0], marker="o")
    axes[1, 0].set_title('Total Points per Gameweek')
    axes[1, 0].set_xlabel('Gameweek')
    axes[1, 0].set_ylabel('Total Points')
    
    # Number of Goals per Gameweek
    sns.barplot(x='Gameweek', y='Goal Points', data=player_data, ax=axes[1, 1], palette="viridis")
    axes[1, 1].set_title('Number of Goals per Gameweek')
    axes[1, 1].set_xlabel('Gameweek')
    axes[1, 1].set_ylabel('Goal Points')
    
    # Pie chart showing Wins, Losses, and Draws
    game_outcomes = player_data['Game Outcome'].value_counts()
    axes[2, 0].pie(game_outcomes, labels=game_outcomes.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
    axes[2, 0].set_title('Game Outcomes: Wins, Losses, Draws')
    
    # Line chart showing Defensive score and Midfield score over Gameweek
    sns.lineplot(x='Gameweek', y='Defensive Score Points', data=player_data, ax=axes[2, 1], label='Defensive Score', marker="o")
    sns.lineplot(x='Gameweek', y='Midfield Score', data=player_data, ax=axes[2, 1], label='Midfield Score', marker="o")
    axes[2, 1].set_title('Defensive and Midfield Score over Gameweek')
    axes[2, 1].set_xlabel('Gameweek')
    axes[2, 1].set_ylabel('Score')
    axes[2, 1].legend()
    
    # Adjust layout
    plt.tight_layout()
    # Return the figure object rather than showing it
    return fig


def save_player_graphs_to_html():
    """
    Loop through all distinct players in the dataset.
    For each player, generate the plots and save them to an HTML file named after the player.
    """
    
    # Create a directory to store the HTML files
    if not os.path.exists("/workspaces/SPL/Middleput/player_data/s1/player_graphs"):
        os.makedirs("/workspaces/SPL/Middleput/player_data/s1/player_graphs")

    # Get the list of distinct players
    players = data['Player'].unique()

    # Loop through each player
    for player in players:
        # Generate the player's plots
        fig = player_graphs_with_summary_updated(player)
        
        # Convert the figure to HTML
        html_content = mpl_to_html(fig)
        
        # Save the plots to an HTML file
        with open(f"/workspaces/SPL/Middleput/player_data/s1/player_graphs/{player}.html", "w") as file:
            file.write(html_content)
        plt.close(fig)

    return f"Graphs saved for {len(players)} players in /workspaces/SPL/Middleput/player_data/s1/player_graphs"

save_player_graphs_to_html()

'Graphs saved for 9 players in /workspaces/SPL/Middleput/player_data/s1/player_graphs'

In [23]:
file_sorted = "/workspaces/SPL/Middleput/sorted_summary.xlsx"
df = pd.read_excel(file_sorted)
# Update the Player column to hyperlink with the new path structure
df['Player'] = df['Player'].apply(lambda x: f'<a href="/SPL/Middleput/player_data/s1/player_graphs/{x}.html">{x}</a>')

# Convert the DataFrame to HTML again
html_string = df.to_html(escape=False, index=False)

# Update the complete HTML structure with the new hyperlinks
html_complete = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Players Summary</title>
</head>
<body>
    {html_string}
</body>
</html>
"""

# Save the updated HTML to a file
output_path_updated = "/workspaces/SPL/Middleput/players_summary_updated.html"
with open(output_path_updated, 'w') as f:
    f.write(html_complete)


# 1. Read the HTML and CSS files
with open("/workspaces/SPL/Middleput/players_summary_updated.html", "r") as html_file:
    players_summary_html = html_file.read()

# 2. Parse the HTML using BeautifulSoup
soup = BeautifulSoup(players_summary_html, 'html.parser')

# 3. Link the CSS file in the HTML
link_tag = soup.new_tag("link")
link_tag.attrs["rel"] = "stylesheet"
link_tag.attrs["href"] = "styles_table.css"
soup.head.append(link_tag)

# 4. Wrap the table in a container div for better control and presentation
container_div = soup.new_tag("div", id="table-container")
soup.table.wrap(container_div)

# 5. Correct the malformed anchor tags
for a_tag in soup.find_all('a'):
    if "<a href=" in a_tag["href"]:
        a_tag.decompose()

# 6. Remove residual `.html"&gt;` entries
for text_element in soup.find_all(text=True):
    if '.html"&gt;' in text_element:
        text_element.replace_with(text_element.replace('.html"&gt;', ''))

# 9. Save the updated HTML and CSS to files
with open("/workspaces/SPL/Output/players_stats_new.html", "w") as html_file:
    html_file.write(str(soup.prettify()))

/tmp/ipykernel_23137/3160401182.py:53: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for text_element in soup.find_all(text=True):
